In [359]:
from pathlib import Path
import numpy as np
import re
from math import prod
from collections import defaultdict, deque
from copy import copy
from itertools import pairwise, permutations, product, combinations

In [360]:
data = Path('../Data/Day24.txt').read_text().splitlines()

In [368]:
wires = {}
gates = {}

for line in data:
    if line.find('->') != -1:
        w1, op, w2, _, out = line.split()
        gates[out] = [w1, w2, op]
    elif line.find(': ') != -1:
        w, v = line.split(': ')
        wires[w] = int(v)

num_bits = len([wire for wire in wires if wire[0] == 'x'])


In [392]:
def get_wire_output(wire, wires = wires, gates = gates, swaps = {}):
    if wire in wires:
        return wires[wire]
    
    if wire not in swaps:
        w1, w2, op = gates[wire]
    else:
        w1, w2, op = gates[swaps[wire]]

    out1 = get_wire_output(w1, wires, gates, swaps)
    out2 = get_wire_output(w2, wires, gates, swaps)

    match op:
        case 'OR':
            wires[wire] = out1 | out2 
        case 'XOR':
            wires[wire] = out1 ^ out2
        case 'AND':
            wires[wire] = out1 * out2
    
    return wires[wire]

def calculate_final_output(wires = wires, gates = gates, swaps = {}):

    for wire in gates:
        get_wire_output(wire, wires, gates, swaps)

    return int(''.join([str(wires[z]) for z in sorted([w for w in wires if w.startswith('z')], reverse = True)]), 2)

def calculate_from_xy(x, y, num_bits = num_bits, swaps = {}):
    wires2 = {}
    for i in range(num_bits):
        wires2[f"x{i:02}"] = (x >> i) % 2
        wires2[f"y{i:02}"] = (y >> i) % 2

    return calculate_final_output(wires2, gates, swaps), wires2

def get_lineage(gate, gates = gates):

    lineage = set()

    nodes = deque([gate])

    while nodes:
        node = nodes.pop()

        lineage.add(node)

        if node in gates:
            w1, w2, _ = gates[node]

            for w in [w1, w2]:
                if w not in lineage:
                    nodes.append(w)

    return lineage

In [393]:
calculate_final_output()

57344080719736

In [394]:
# Ideas

# 0 for both gates returns 0 regardless of operation
# We can select our inputs so only a few gate operations are significant
# Swaps only matter if we swap 0's with 1's
# Some swaps cause infinite loops

In [482]:
# We noticed that there are four distinct cases that each require at least one swap
# Therefore, we can get our four swaps by taking one good swap from each case
for i in range(45):
    res, output_wires = calculate_from_xy(2**i, 0)

    if res != 2**i:
        print(i, {wire for wire,value in output_wires.items() if value == 1})

5 {'nbc', 'pdf', 'z06', 'x05'}
15 {'kqk', 'z16', 'fwr', 'x15'}
23 {'z24', 'cgq', 'hpw', 'x23', 'ngq'}
39 {'fnr', 'z40', 'bdr', 'x39', 'sbn'}


In [483]:
# Same as prior, but now we look for good swaps for each case

lineages = {gate:get_lineage(gate) for gate in gates}
swap_candidates = []
for i in range(45):
    res, output_wires = calculate_from_xy(2**i, 0)
    if  res != 2**i:
        zero_gates = []
        one_gates = []
        for gate in gates:
            if output_wires[gate] == 0:
                zero_gates.append(gate)
            else:
                one_gates.append(gate)
        
        good_swaps = []
        for g1, g2 in product(one_gates, zero_gates):
            if g2 in lineages[g1] or g1 in lineages[g2]:
                continue
            swaps = {
                g1: g2,
                g2: g1
            }

            swap_res, _ = calculate_from_xy(2**i, 0, swaps = swaps)

            if swap_res == 2**i:
                good_swaps.append(swaps)
        
        swap_candidates.append(good_swaps)

In [484]:
# Test possible swap candidates

tests = [
    [2**45-1, 2**45-1],
]

tests += [[2**i, 0] for i in range(45)]
tests += [[0, 2**i] for i in range(45)]

tests.append([int('10'*22, 2), int('01'*22, 2)])

tests.append([int(''.join(str(wires[f"{x}{i:02}"]) for i in range(44, -1, -1)), 2) for x in 'xy'])

good_candidates = set()

for candidate in product(*swap_candidates):
    swaps = {k:v for swap in candidate for k,v in swap.items()}

    good_swap = True
    
    for x,y in tests:
        if calculate_from_xy(x, y, swaps = swaps)[0] != x+y:
            good_swap = False
            break
    
    if good_swap:
        good_candidates.add(','.join(sorted(swaps.keys())))

print(len(good_candidates))

2


In [485]:
# Down to 2 candidates, good enough!

good_candidates

{'cgq,fnr,kqk,nbc,svm,z15,z23,z39', 'fnr,kqk,nbc,ngq,svm,z15,z23,z39'}